In [1]:
%matplotlib notebook
%matplotlib inline

import os
import time
import datetime as dt
import numpy as np
from numpy import newaxis
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader
import keras

Using TensorFlow backend.


In [2]:
#mec = graphlab.SFrame('data/sppmecdata.csv')

fname = "data/sppmecdata.csv"
coln = ['interval','hour','windforecast','loadforecast','natgasprice','diffmec','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
mec = pd.read_csv(fname)
mec.head()

,interval,hour,windforecast,loadforecast,natgasprice,diffmec,0,1,2,3,...,14,15,16,17,18,19,20,21,22,23
0,2016-08-01 00:00:00,0.0,9419.15,34925.0,2.97,5.5041,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-08-01 01:00:00,1.0,9526.57,33084.0,2.97,6.5692,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-08-01 02:00:00,2.0,9508.95,31562.0,2.97,3.9592,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-08-01 03:00:00,3.0,9376.61,30472.0,2.97,2.8883,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-08-01 04:00:00,4.0,9228.14,29909.0,2.97,3.2920,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
mec['buysig'] = mec['diffmec'] < 0
mec['interval'] = pd.to_datetime(mec['interval'])
mec['weekday'] = mec['interval'].dt.dayofweek
mec_nolg = mec[abs(mec['diffmec']) < 100]
mec = mec.set_index(['interval'])
mec.tail()

,hour,windforecast,loadforecast,natgasprice,diffmec,0,1,2,3,4,...,16,17,18,19,20,21,22,23,buysig,weekday
interval,,,,,,,,,,,,,,,,,,,,,
2017-03-01 01:00:00,1.0,11282.88,23646.0,2.54,1.9541,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,2
2017-03-01 02:00:00,2.0,10766.69,23273.0,2.54,-0.2795,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,2
2017-03-01 03:00:00,3.0,10275.86,23197.0,2.54,11.9349,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,2
2017-03-01 04:00:00,4.0,9915.80,23384.0,2.54,7.5464,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,2
2017-03-01 05:00:00,5.0,9694.18,23980.0,2.54,1.3101,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,2


In [ ]:
graphlab.canvas.set_target('ipynb')

In [4]:
#trdata,tedata = mec.random_split(.8,seed=0)
#traindata,testdata = mec_nolg.random_split(.8,seed=0)

# training data
prediction_time = 1 #day
testdatasize = int(len(mec)*.1)
unroll_length = 48
testdatacut = testdatasize + unroll_length  + 1

x_train = mec[0:-prediction_time-testdatacut].as_matrix()
y_train = mec[prediction_time:-testdatacut]['diffmec'].as_matrix()

# test data
x_test = mec[0-testdatacut:-prediction_time].as_matrix()
y_test = mec[prediction_time-testdatacut:]['diffmec'].as_matrix()

In [5]:
def unroll(data,sequence_length=24):
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)


x_train = unroll(x_train,unroll_length)
x_test  = unroll(x_test,unroll_length)
y_train = y_train[-x_train.shape[0]:]
y_test  = y_test[-x_test.shape[0]:]


print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (4488, 48, 31)
y_train (4488,)
x_test (509, 48, 31)
y_test (509,)


In [6]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
#import lstm, time #helper libraries

In [7]:
#Step 2 Build Model
model = Sequential()

model.add(LSTM(
    input_dim=x_train.shape[-1],
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print('compilation time : {}'.format(time.time() - start))

compilation time : 0.010058403015136719


In [ ]:
#Step 3 Train the model
model.fit(
    x_train,
    y_train,
    batch_size=4096,
    nb_epoch=256,
    validation_split=0.05)

Train on 4263 samples, validate on 225 samples
Epoch 1/256
4263/4263 [==============================] - 14s - loss: 291.0325 - val_loss: 157.5555
Epoch 2/256
4263/4263 [==============================] - 13s - loss: 289.1622 - val_loss: 156.3116
Epoch 3/256
4263/4263 [==============================] - 14s - loss: 287.6665 - val_loss: 156.3595
Epoch 4/256
4263/4263 [==============================] - 12s - loss: 287.7343 - val_loss: 157.2396
Epoch 5/256
4263/4263 [==============================] - 13s - loss: 288.1813 - val_loss: 157.2883
Epoch 6/256
4263/4263 [==============================] - 14s - loss: 287.9090 - val_loss: 157.2977
Epoch 7/256
4263/4263 [========================

In [ ]:
sppfeatures = ['loadforecast','windforecast','natgasprice','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
mec[sppfeatures].show()

In [ ]:
mec.show(view='BoxWhisker Plot', x='hour', y='diffmec')

In [ ]:
spp_nolg_regression_model = graphlab.linear_regression.create(traindata,target='diffmec',features=sppfeatures,validation_set=testdata)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(mec_nolg['loadforecast'],mec_nolg['diffmec'],'.',
        mec_nolg['loadforecast'],spp_nolg_regression_model.predict(mec_nolg),'-')

In [ ]:
spp_regression_model = graphlab.linear_regression.create(trdata,target='diffmec',features=sppfeatures,validation_set=tedata)

In [ ]:
plt.plot(mec['loadforecast'],mec['diffmec'],'.',
        mec['loadforecast'],spp_regression_model.predict(mec),'-')
plt.ylim(-30,30)
plt.show()

In [ ]:
coeff = spp_regression_model.get("coefficients")
print coeff

In [ ]:
print spp_regression_model.evaluate(tedata)

In [ ]:
tint = {'windforecast':[7500],
        'loadforecast':[28777],
        'natgasprice':[3.20],
        '0':[0],
        '1':[0],
        '2':[0],
        '3':[0],
        '4':[0],
        '5':[0],
        '6':[0],
        '7':[0],
        '8':[1],
        '9':[0],
        '10':[0],
        '11':[0],
        '12':[0],
        '13':[0],
        '14':[0],
        '15':[0],
        '16':[0],
        '17':[0],
        '18':[0],
        '19':[0],
        '20':[0],
        '21':[0],
        '22':[0],
        '23':[0]}

print spp_regression_model.predict(graphlab.SFrame(tint))

In [ ]:


spp_classifier_model = graphlab.logistic_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

In [ ]:

spp_nolg_classifier_model = graphlab.logistic_classifier.create(traindata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=testdata)

In [ ]:

spp_nn_model = graphlab.neuralnet_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

In [ ]:
spp_classifier_model.evaluate(tedata, metric='roc_curve')

In [ ]:
spp_classifier_model.show(view='Evaluation')

In [ ]:
mec['predicted_buysig'] = spp_classifier_model.predict(mec, output_type='probability')
reg = spp_regression_model.predict(graphlab.SFrame(tint))
cla = spp_classifier_model.predict(graphlab.SFrame(tint))
#nn = spp_nn_model.predict(graphlab.SFrame(tint))
print 'Regression model predicts diffmec of ' + str(reg)
print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
#print 'NN model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(nn)

In [2]:
def create_hour(w,l,n,hour):
    return {'windforecast':[w],
        'loadforecast':[l],
        'natgasprice':[n],
        str(hour):[1]}
    

In [3]:
#3/31/2017 pred
wf = [5130,5295,5260,5140,5046,5000,4924,4814,4797,4763,4765,5080,5543,5952,6294,6528,6767,7132,7586,7993,8335,8658,8919,8970]
lf = [24510,23501,22994,22768,22780,23261,24618,26973,28441,28467,28337,28266,28090,27834,27725,27585,27231,27284,27124,26951,26965,27327,27327,26701,25464]
ng = 3.20
for h in range(0,24):
    reg = spp_regression_model.predict(graphlab.SFrame(create_hour(wf[h],lf[h],ng,h)))
    cla = spp_classifier_model.predict(graphlab.SFrame(create_hour(wf[h],lf[h],ng,h)))
    
    if reg[0] < -2 and cla[0] == 1:
        print 'Hour ' + str(h)
        print '---------------------'
        print 'STRONG BUY'
        print 'Regression model predicts diffmec of ' + str(reg)
        print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
        print ''
    elif reg[0] > 2 and cla[0] == 0:
        print 'Hour ' + str(h)
        print '---------------------'
        print 'STRONG SELL'
        print 'Regression model predicts diffmec of ' + str(reg)
        print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
        print ''
    #else:
        #print 'Hour ' + str(h)
        #print '---------------------'
        #print 'MIXED'
        #print ''

NameError: name 'spp_regression_model' is not defined